# Advanced Exploratory Data Analysis of Brain Tumor Data

## Import dependencies

In [ ]:
import os
import ast
import json
import glob
import random
import collections
from tqdm import tqdm
import gc

# Visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

# Seed for reproducability
seed = 1234
np.random.seed(seed)

## Quick EDA and data Visualization

In [ ]:
# Paths 
KAGGLE_DIR = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
IMG_PATH_TRAIN = KAGGLE_DIR + 'train/'
IMG_PATH_TEST = KAGGLE_DIR + 'test/'
TRAIN_CSV_PATH = KAGGLE_DIR + 'train_labels.csv'
TEST_CSV_PATH = KAGGLE_DIR + 'sample_submission.csv'

In [ ]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
display(train_df.head(5))
print('MGMT counts:')
train_df.MGMT_value.value_counts()

In [ ]:
plt.figure(figsize=(5, 5))
plt.title('Train csv')
sns.countplot(data=train_df, x="MGMT_value");

In [ ]:
test_df = pd.read_csv(TEST_CSV_PATH)
display(test_df.head(5))
print('MGMT counts:')
test_df.MGMT_value.value_counts()

In [ ]:
plt.figure(figsize=(5, 5))
plt.title('Test csv')
sns.countplot(data=test_df, x="MGMT_value");

In [ ]:
# All filenames for train and test images
train_images = os.listdir(IMG_PATH_TRAIN)
test_images = os.listdir(IMG_PATH_TEST)

In [ ]:
def load_dicom(path):
    # read file
    dicom = pydicom.read_file(path)
    # get pixel data into a useful format. 
    data = dicom.pixel_array
    # transform data into black and white scale / grayscale
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data


def visualize_sample(
    brats21id, 
    slice_i,
    mgmt_value,
    types=("FLAIR", "T1w", "T1wCE", "T2w")
):
    plt.figure(figsize=(16, 5))
    patient_path = os.path.join(
        IMG_PATH_TRAIN, 
        str(brats21id).zfill(5),
    )
    for i, t in enumerate(types, 1):
        t_paths = sorted(
            glob.glob(os.path.join(patient_path, t, "*")), 
            key=lambda x: int(x[:-4].split("-")[-1]),
        )
        data = load_dicom(t_paths[int(len(t_paths) * slice_i)])
        plt.subplot(1, 4, i)
        plt.imshow(data, cmap="gray")
        plt.title(f"{t}", fontsize=16)
        plt.axis("off")

    plt.suptitle(f"MGMT_value: {mgmt_value}", fontsize=16)
    plt.show()

In [ ]:
for i in random.sample(range(train_df.shape[0]), 10): # get 10 random indexes from the train ds
    _brats21id = train_df.iloc[i]["BraTS21ID"] # for these indexes get the associated brats ID
    _mgmt_value = train_df.iloc[i]["MGMT_value"] # and tumor class
    visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5) # visualize samples

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_animation(ims):
    fig = plt.figure(figsize=(6, 6))
    plt.axis('off')
    im = plt.imshow(ims[0], cmap="gray")

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

In [ ]:
def load_dicom_line(path):
    t_paths = sorted(
        glob.glob(os.path.join(path, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    images = []
    for filename in t_paths:
        data = load_dicom(filename)
        if data.max() == 0:
            continue
        images.append(data)
        
    return images

## EDA of DICOM files

In [ ]:
IMG_PATH_TRAIN = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/"
IMG_PATH_TEST = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/"

In [ ]:
# review training directory
s_sizes = [] # list of no. of scans present for each patient
p_sizes = [] # list of no. of dcm files present for each patient
patient_id = [] # patient id
file_paths = [] # file_paths

for d in os.listdir(IMG_PATH_TRAIN):
#     print("Patient '{}' has {} scans and a total of {} DICOM images".format(d, len(os.listdir(TRAIN_DIR + d)), len(glob.glob(TRAIN_DIR+ d + "/*/*.dcm"))))
    s_sizes.append(len(os.listdir(IMG_PATH_TRAIN + d)))
    p_sizes.append(len(glob.glob(IMG_PATH_TRAIN + d + "/*/*.dcm")))
    patient_id.append(d)

patient_files_df = pd.DataFrame(
    {'patient_id': patient_id,
     'file_count': p_sizes,
    })
    
print('----')
print('Total patients {} Total DCM files {}'.format(len(os.listdir(IMG_PATH_TRAIN)), 
                                                      len(glob.glob(IMG_PATH_TRAIN+ "/*/*/*.dcm"))))

print('----')
print('TRAIN Dataframe with File Count per Patient ')
display(patient_files_df.head(5))

print('----')
print('Verify total File Count for all Patients ')
print('Total number of patients:', patient_files_df.shape[0])

print('Total file count:', patient_files_df.file_count.sum())

In [ ]:
f = []
for (dirpath, dirnames, filenames) in os.walk(IMG_PATH_TRAIN):
    f.extend(os.path.join(dirpath, x) for x in filenames)
    
train_file_paths_df = pd.DataFrame({'file_paths': f})
train_file_paths_df['train_dir'] = IMG_PATH_TRAIN
train_file_paths_df['patient_id'] = train_file_paths_df['file_paths'].str.split("/", n = 7, expand = True)[4]
train_file_paths_df['scan_type'] = train_file_paths_df['file_paths'].str.split("/", n = 7, expand = True)[5]
train_file_paths_df['file'] = train_file_paths_df['file_paths'].str.split("/", n = 7, expand = True)[6]
display(train_file_paths_df.head(2))
train_file_paths_df.shape[0]

In [ ]:
print('Possible Number of scans for all patients:', set(s_sizes))

In [ ]:
# lets visualize trainig data
p = sns.color_palette()
plt.hist(p_sizes, color=p[2])
plt.ylabel('Number of patients')
plt.xlabel('Count of DICOM files')
plt.title('Histogram of DICOM count per patient - Training Data');

In [ ]:
# review test directory
s_sizes = [] # list of no. of scans present for each patient
p_sizes = [] # list of no. of dcm files present for each patient
patient_id = [] # patient id

for d in os.listdir(IMG_PATH_TEST):
#     print("Patient '{}' has {} scans and a total of {} DICOM images".format(d, 
#                     len(os.listdir(IMG_PATH_TEST + d)), len(glob.glob(IMG_PATH_TEST+ d + "/*/*.dcm"))))
    s_sizes.append(len(os.listdir(IMG_PATH_TEST + d)))
    p_sizes.append(len(glob.glob(IMG_PATH_TEST + d + "/*/*.dcm")))
    patient_id.append(d)

patient_files_df = pd.DataFrame(
    {'patient_id': patient_id,
     'file_count': p_sizes,
    })
    
print('----')
print('Total patients {} Total DCM files {}'.format(len(os.listdir(IMG_PATH_TEST)), 
                                                      len(glob.glob(IMG_PATH_TEST+ "/*/*/*.dcm"))))
print('----')
print('TRAIN Dataframe with File Count per Patient ')
display(patient_files_df.head(5))

print('----')
print('Verify total File Count for all Patients ')
print('Total number of patients:', patient_files_df.shape[0])

print('Total file count:', patient_files_df.file_count.sum())

In [ ]:
f = []
for (dirpath, dirnames, filenames) in os.walk(IMG_PATH_TEST):
    f.extend(os.path.join(dirpath, x) for x in filenames)
    
test_file_paths_df = pd.DataFrame({'file_paths': f})
test_file_paths_df['train_dir'] = IMG_PATH_TEST
test_file_paths_df['patient_id'] = test_file_paths_df['file_paths'].str.split("/", n = 7, expand = True)[4]
test_file_paths_df['scan_type'] = test_file_paths_df['file_paths'].str.split("/", n = 7, expand = True)[5]
test_file_paths_df['file'] = test_file_paths_df['file_paths'].str.split("/", n = 7, expand = True)[6]
display(test_file_paths_df.head(2))
test_file_paths_df.shape[0]

In [ ]:
print('Possible Number of scans for all patients:', set(s_sizes))

In [ ]:
# lets visualize test data
p = sns.color_palette()
plt.hist(p_sizes, color=p[2])
plt.ylabel('Number of patients')
plt.xlabel('Count of DICOM files')
plt.title('Histogram of DICOM count per patient - Training Data');

## EDA of DICOM Metadata

In [ ]:
# All columns for which we want to collect information
meta_cols = ['SpecificCharacterSet','ImageType','SOPClassUID',
             'SOPInstanceUID','AccessionNumber','Modality', 'SeriesDescription', 
             'PatientID', 'MRAcquisitionType', 'SliceThickness', 
             'EchoTime', 'NumberOfAverages', 'ImagingFrequency', 'ImagedNucleus', 
             'MagneticFieldStrength', 'SpacingBetweenSlices', 
             'EchoTrainLength', 'PercentSampling', 'PercentPhaseFieldOfView',
             'PixelBandwidth', 'TriggerWindow', 'ReconstructionDiameter', 'AcquisitionMatrix',
             'FlipAngle', 'SAR', 'PatientPosition',
             'StudyInstanceUID', 'SeriesInstanceUID', 'SeriesNumber', 'InstanceNumber',
             'ImagePositionPatient', 'ImageOrientationPatient', 'Laterality',
             'PositionReferenceIndicator', 'SliceLocation', 'InStackPositionNumber',
             'SamplesPerPixel', 'PhotometricInterpretation', 'Rows', 'Columns', 'PixelSpacing',
             'BitsAllocated', 'BitsStored', 'HighBit', 'PixelRepresentation', 'WindowCenter',
             'WindowWidth', 'RescaleIntercept', 'RescaleSlope', 'RescaleType']

In [ ]:
# Initialize dictionaries to collect the metadata
col_dict_train = {col: [] for col in meta_cols}
col_dict_test = {col: [] for col in meta_cols}

In [ ]:
test_meta_df = pd.read_csv("../input/stage0-metadata-rsna/stage_0_test_with_metadata.csv")
train_meta_df = pd.read_csv("../input/stage0-metadata-rsna/stage_0_train_with_metadata.csv")

In [ ]:
meta_attr = []
num_unique = []

for col in train_meta_df:
#     print("* For attribute  '{}' , there are [ {} ] unique values.".format(col,
#                     len(train_meta_df[col].unique())))
    meta_attr.append(col)
    num_unique.append(len(train_meta_df[col].unique()))
    
train_meta_values_df = pd.DataFrame(
    {'attribute': meta_attr,
     'value_count': num_unique,
     'nan_count': train_meta_df.isna().sum()
    })

train_meta_values_df = train_meta_values_df.sort_values(by=['value_count'], ascending=False).reset_index(drop=True)
train_meta_values_df

In [ ]:
meta_attr = []
num_unique = []

for col in test_meta_df:
#     print("* For attribute  '{}' , there are [ {} ] unique values.".format(col,
#                     len(train_meta_df[col].unique())))
    meta_attr.append(col)
    num_unique.append(len(test_meta_df[col].unique()))
    
test_meta_values_df = pd.DataFrame(
    {'attribute': meta_attr,
     'value_count': num_unique,
     'nan_count': test_meta_df.isna().sum()
    })

test_meta_values_df = test_meta_values_df.sort_values(by=['value_count'], ascending=False).reset_index(drop=True)
test_meta_values_df

In [ ]:
def color_code_by_vcount(df):
    if df['value_count'] == 1.0:
        return 'k' # Single unique value, color-code black
    elif df['value_count'] <= 1000.0:
        return 'b' # Unique value count between one and 1000, color-code blue
    else:
        return 'r' # Unique value count more than > 1000, color-code red

train_mv_df = train_meta_values_df.copy().set_index("attribute")
train_mv_df['color'] = train_mv_df.apply(color_code_by_vcount, axis=1)

ax = train_mv_df['value_count'].plot(kind='bar',
                                    figsize=(14,8),  color=train_mv_df['color'],
                                    title="Number of Unique Values per Attribute [LOG SCALE]")
ax.set_xlabel("Metadata Attribute")
ax.set_ylabel("Unique Number of Values [LOG SCALE]")
ax.set_yscale('log');

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(15,15))

nans = train_meta_df.isna().sum().sort_values(ascending=False)
sns.barplot(y=nans.index, x=nans, orient='h', ax = axes[0])
axes[0].set_title("Train NaN Count")

nans_test = test_meta_df.isna().sum().sort_values(ascending=False)
sns.barplot(y=nans_test.index, x=nans_test, orient='h', ax = axes[1])
axes[1].yaxis.set_ticks_position("right")
axes[1].set_title("Test NaN Count");

In [ ]:
merged_meta_attrs = pd.merge(train_meta_values_df, test_meta_values_df, on="attribute", 
                             suffixes=("_train","_test"))
merged_meta_attrs

In [ ]:
'''Print attributes unique values for low value count and descrepancy between test and train.'''
print(train_meta_df["SpecificCharacterSet"].unique())
print(test_meta_df["PositionReferenceIndicator"].unique())

In [ ]:
list_drop_attrs = ["SOPInstanceUID","ImagePositionPatient","SliceLocation","WindowWidth",
                   "WindowCenter","SeriesInstanceUID","SAR","ImageOrientationPatient",
                   "ImagingFrequency","AccessionNumber","StudyInstanceUID","HighBit",
                   "RescaleIntercept","BitsStored","BitsAllocated","RescaleSlope",
                   "PatientPosition","PhotometricInterpretation","SamplesPerPixel",
                   "PositionReferenceIndicator","Laterality",'ImageType',"SpacingBetweenSlices",
                   "Modality","SOPClassUID","RescaleType",]

In [ ]:
print("Columns before : ", len(train_meta_df.columns))
train_meta_df_useful = train_meta_df.drop(list_drop_attrs, axis=1)
print("Columns after : ", len(train_meta_df_useful.columns))

In [ ]:
sizes = train_meta_df.apply(lambda x: f'{x.Rows}x{x.Columns}', axis=1)
plt.figure(figsize=(15, 8))
plt.xticks(rotation=45)
sns.countplot(sizes);

In [ ]:
sns.countplot(train_meta_df.SeriesDescription);

In [ ]:
plt.figure(figsize=(12, 5))
plt.xticks(rotation=45)
sns.countplot(train_meta_df.SliceThickness);

In [ ]:
sns.jointplot(data=train_meta_df, x='SliceThickness', y='SpacingBetweenSlices');

In [ ]:
sns.countplot(train_meta_df.NumberOfAverages);

In [ ]:
sns.countplot(train_meta_df.MagneticFieldStrength);

In [ ]:
plt.figure(figsize=(12, 5))
plt.xticks(rotation=45)
sns.countplot(train_meta_df.ReconstructionDiameter);

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))   
sns.heatmap(train_meta_df.corr(), ax =ax)

## EDA of pixel data

In [ ]:
def image_stats(image):
    nonzero_pixels = image[np.nonzero(image)]
    if nonzero_pixels.shape == (0,):
        mean = 0
        std = 0
    else:
        mean = np.mean(nonzero_pixels)
        std = np.std(nonzero_pixels)
    return (mean,std)

def plot_image_hist(image, threshold = 1.5, normalize = False):
    pixels = image.ravel()
    nonzero_pixels = pixels[np.nonzero(pixels)]
    (mean,std) = image_stats(nonzero_pixels)
    if normalize:
        nonzero_pixels = (nonzero_pixels - mean) / std
        (mean,std) = image_stats(nonzero_pixels)
    over_threshold = np.count_nonzero(nonzero_pixels > mean + threshold * std)

    fig, (axi, axh) = plt.subplots(1, 2, figsize = (20,3), 
                                   gridspec_kw={'width_ratios': [1, 4]})
    fig.suptitle(f'Pixels over threshold # ({over_threshold})')

    axh.hist(nonzero_pixels, 200)

    ax_limits = axh.get_ylim()
    axh.vlines(mean, ymin=ax_limits[0], 
               ymax=ax_limits[1], colors='b')
    axh.vlines(mean+std, ymin=ax_limits[0], 
               ymax=ax_limits[1], colors='b', linestyles='dotted')
    axh.vlines(mean + threshold * std, ymin=ax_limits[0], 
               ymax=ax_limits[1], colors='b', linestyles='dashed')
    axi.imshow(image, cmap = plt.cm.gray)
    axi.grid(False)
    axi.axis('off')
    plt.show()

In [ ]:
img = load_dicom("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T2w/Image-200.dcm")
plot_image_hist(img, threshold = 2, normalize = True)

In [ ]:
plot_image_hist(img, threshold = 2, normalize = False)

In [ ]:
def visualize_hist_sample_image(
    brats21id, 
    slice_i,
    mgmt_value,
    types=("FLAIR", "T1w", "T1wCE", "T2w"),
    threshold = 1.5, normalize = False
):
    plt.figure(figsize=(16, 5))
    patient_path = os.path.join(
        IMG_PATH_TRAIN, 
        str(brats21id).zfill(5),
    )
    for i, t in enumerate(types, 1):
        t_paths = sorted(
            glob.glob(os.path.join(patient_path, t, "*")), 
            key=lambda x: int(x[:-4].split("-")[-1]),
        )
        image = load_dicom(t_paths[int(len(t_paths) * slice_i)])
        pixels = image.ravel()
        nonzero_pixels = pixels[np.nonzero(pixels)]
        (mean,std) = image_stats(nonzero_pixels)
        if normalize:
            nonzero_pixels = (nonzero_pixels - mean) / std
            (mean,std) = image_stats(nonzero_pixels)
        over_threshold = np.count_nonzero(nonzero_pixels > mean + threshold * std)

        fig, (axi, axh) = plt.subplots(1, 2, figsize = (20,3), 
                                       gridspec_kw={'width_ratios': [1, 4]});

        axh.hist(nonzero_pixels, 200)

        ax_limits = axh.get_ylim()
        axh.vlines(mean, ymin=ax_limits[0], 
                   ymax=ax_limits[1], colors='b')
        axh.vlines(mean+std, ymin=ax_limits[0], 
                   ymax=ax_limits[1], colors='b', linestyles='dotted')
        axh.vlines(mean + threshold * std, ymin=ax_limits[0], 
                   ymax=ax_limits[1], colors='b', linestyles='dashed')
        axi.imshow(image, cmap = plt.cm.gray)
        axi.grid(False)
        plt.title(f"{t}: pixels over threshold # ({over_threshold})", fontsize=16)
        axi.axis('off')
#     plt.suptitle(f"MGMT_value: {mgmt_value}", fontsize=16)
    plt.show()
    print(f"MGMT_value: {mgmt_value}")

In [ ]:
for i in random.sample(range(train_df.shape[0]), 2): # get 10 random indexes from the train ds
    _brats21id = train_df.iloc[i]["BraTS21ID"] # for these indexes get the associated brats ID
    _mgmt_value = train_df.iloc[i]["MGMT_value"] # and tumor class
    visualize_hist_sample_image(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5,
                           threshold = 2, normalize = True) # visualize samples

In [ ]:
def visualize_masked_sample(
    brats21id, 
    slice_i,
    mgmt_value,
    types=("FLAIR", "T1w", "T1wCE", "T2w"),
    threshold = -1
):
    plt.figure(figsize=(16, 5))
    patient_path = os.path.join(
        IMG_PATH_TRAIN, 
        str(brats21id).zfill(5),
    )
    for i, t in enumerate(types, 1):
        t_paths = sorted(
            glob.glob(os.path.join(patient_path, t, "*")), 
            key=lambda x: int(x[:-4].split("-")[-1]),
        )
        data = load_dicom(t_paths[int(len(t_paths) * slice_i)])
        if threshold > -1:
            data[data < threshold] = 0
        plt.subplot(1, 4, i)
        plt.imshow(data, cmap="gray")
        plt.title(f"{t}", fontsize=16)
        plt.axis("off")

    plt.suptitle(f"MGMT_value: {mgmt_value}", fontsize=16)
    plt.show()

In [ ]:
i = 520
_brats21id = train_df.iloc[i]["BraTS21ID"] # for these indexes get the associated brats ID
_mgmt_value = train_df.iloc[i]["MGMT_value"] # and tumor class
visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5) # visualize samples

In [ ]:
i = 520
_brats21id = train_df.iloc[i]["BraTS21ID"] # for these indexes get the associated brats ID
_mgmt_value = train_df.iloc[i]["MGMT_value"] # and tumor class
visualize_masked_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.4,
                           threshold = 80) # visualize samples

In [ ]:
i = 520
_brats21id = train_df.iloc[i]["BraTS21ID"] # for these indexes get the associated brats ID
_mgmt_value = train_df.iloc[i]["MGMT_value"] # and tumor class
visualize_masked_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5,
                           threshold = 120) # visualize samples

In [ ]:
train_px_df = pd.read_csv('/kaggle/input/train-test-filepaths-rsna-full/stats_train_file_paths_df.csv')
test_px_df = pd.read_csv('/kaggle/input/train-test-filepaths-rsna-full/stats_test_file_paths_df.csv')

In [ ]:
stats_cols = []
num_unique = []

for col in train_px_df:
#     print("* For attribute  '{}' , there are [ {} ] unique values.".format(col,
#                     len(train_meta_df[col].unique())))
    stats_cols.append(col)
    num_unique.append(len(train_px_df[col].unique()))
    
train_df_stats = pd.DataFrame(
    {'col_name': stats_cols,
     'value_count': num_unique,
     'nan_count': train_px_df.isna().sum()
    })

train_df_stats = train_df_stats.sort_values(by=['value_count'], ascending=False).reset_index(drop=True)
train_df_stats = train_df_stats.set_index('col_name').T
train_df_stats

In [ ]:
stats_cols = []
num_unique = []

for col in test_px_df:
#     print("* For attribute  '{}' , there are [ {} ] unique values.".format(col,
#                     len(train_meta_df[col].unique())))
    stats_cols.append(col)
    num_unique.append(len(test_px_df[col].unique()))
    
test_df_stats = pd.DataFrame(
    {'col_name': stats_cols,
     'value_count': num_unique,
     'nan_count': test_px_df.isna().sum()
    })

test_df_stats = test_df_stats.sort_values(by=['value_count'], ascending=False).reset_index(drop=True)
test_df_stats = test_df_stats.set_index('col_name').T
test_df_stats

In [ ]:
print(train_px_df.min_px.unique(), train_px_df.max_px.unique())
print(test_px_df.min_px.unique(), test_px_df.max_px.unique())

In [ ]:
min(train_px_df.mean_px.unique()), max(train_px_df.mean_px.unique())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(25, 8), sharey=True)
fig.suptitle('Train - Test Dataset Pixel Distributions Mean + STD')

sns.histplot(ax=axes[0], data = train_px_df[['mean_px', 'std_px']], bins=50, alpha=0.5)
axes[0].set_yscale('log')
sns.histplot(ax=axes[1], data = test_px_df[['mean_px', 'std_px']], bins=50, alpha=0.5)
plt.show();

## 3D visualizations

In [ ]:
images = load_dicom_line(IMG_PATH_TRAIN + "00000/FLAIR")
create_animation(images)

In [ ]:
images = load_dicom_line(IMG_PATH_TRAIN + "00000/T1w")
create_animation(images)

In [ ]:
images = load_dicom_line(IMG_PATH_TRAIN + "00000/T1wCE")
create_animation(images)

In [ ]:
images = load_dicom_line(IMG_PATH_TRAIN + "00000/T2w")
create_animation(images)